In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Chronic_kidney_disease"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Chronic_kidney_disease/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Chronic_kidney_disease/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Chronic_kidney_disease/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Chronic_kidney_disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
import os

# Check if there's a suitable cohort directory for Chronic_kidney_disease
print(f"Looking for a relevant cohort directory for {trait}...")

# Check available cohorts
available_dirs = os.listdir(tcga_root_dir)
print(f"Available cohorts: {available_dirs}")

# Kidney disease-related keywords
kidney_keywords = ['kidney', 'renal', 'nephro', 'kich', 'kirc', 'kirp']

# Look for Kidney disease-related directories
kidney_related_dirs = []
for d in available_dirs:
    if any(keyword in d.lower() for keyword in kidney_keywords):
        kidney_related_dirs.append(d)

print(f"Kidney disease-related cohorts: {kidney_related_dirs}")

if not kidney_related_dirs:
    print(f"No suitable cohort found for {trait}.")
    # Mark the task as completed by recording the unavailability
    validate_and_save_cohort_info(
        is_final=False,
        cohort="TCGA",
        info_path=json_path,
        is_gene_available=False,
        is_trait_available=False
    )
    # Exit the script early since no suitable cohort was found
    selected_cohort = None
else:
    # Since we're looking for chronic kidney disease specifically, prioritize 
    # directories that might be more relevant to this specific condition
    # For now, we'll take all matches as they're all kidney-related cancers
    selected_cohort = kidney_related_dirs[0]  # We'll use the first match if multiple exist

if selected_cohort:
    print(f"Selected cohort: {selected_cohort}")
    
    # Get the full path to the selected cohort directory
    cohort_dir = os.path.join(tcga_root_dir, selected_cohort)
    
    # Get the clinical and genetic data file paths
    clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)
    
    print(f"Clinical data file: {os.path.basename(clinical_file_path)}")
    print(f"Genetic data file: {os.path.basename(genetic_file_path)}")
    
    # Load the clinical and genetic data
    clinical_df = pd.read_csv(clinical_file_path, index_col=0, sep='\t')
    genetic_df = pd.read_csv(genetic_file_path, index_col=0, sep='\t')
    
    # Print the column names of the clinical data
    print("\nClinical data columns:")
    print(clinical_df.columns.tolist())
    
    # Basic info about the datasets
    print(f"\nClinical data shape: {clinical_df.shape}")
    print(f"Genetic data shape: {genetic_df.shape}")


Looking for a relevant cohort directory for Chronic_kidney_disease...
Available cohorts: ['TCGA_Liver_Cancer_(LIHC)', 'TCGA_Lower_Grade_Glioma_(LGG)', 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)', 'TCGA_Lung_Adenocarcinoma_(LUAD)', 'TCGA_Lung_Cancer_(LUNG)', 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)', 'TCGA_Melanoma_(SKCM)', 'TCGA_Mesothelioma_(MESO)', 'TCGA_Ocular_melanomas_(UVM)', 'TCGA_Ovarian_Cancer_(OV)', 'TCGA_Pancreatic_Cancer_(PAAD)', 'TCGA_Pheochromocytoma_Paraganglioma_(PCPG)', 'TCGA_Prostate_Cancer_(PRAD)', 'TCGA_Rectal_Cancer_(READ)', 'TCGA_Sarcoma_(SARC)', 'TCGA_Stomach_Cancer_(STAD)', 'TCGA_Testicular_Cancer_(TGCT)', 'TCGA_Thymoma_(THYM)', 'TCGA_Thyroid_Cancer_(THCA)', 'TCGA_Uterine_Carcinosarcoma_(UCS)', '.DS_Store', 'CrawlData.ipynb', 'TCGA_Acute_Myeloid_Leukemia_(LAML)', 'TCGA_Adrenocortical_Cancer_(ACC)', 'TCGA_Bile_Duct_Cancer_(CHOL)', 'TCGA_Bladder_Cancer_(BLCA)', 'TCGA_Breast_Cancer_(BRCA)', 'TCGA_Cervical_Cancer_(CESC)', 'TCGA_Colon_and_Rectal_Cancer_(COADR


Clinical data columns:
['_INTEGRATION', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'additional_surgery_locoregional_procedure', 'additional_surgery_metastatic_procedure', 'age_at_initial_pathologic_diagnosis', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'clinical_M', 'days_to_additional_surgery_metastatic_procedure', 'days_to_birth', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_after_initial_treatment', 'eastern_cancer_oncology_group', 'followup_case_report_form_submission_reason', 'followup_treatment_success', 'form_completion_date', 'gender', 'hemoglobin_result', 'histological_type', 'history_of_neoadjuvant_treatment', 'icd_10', 'icd_o_3_histology', 'icd_o_3_site', 'informed_consent_verified', 'intermediate_dimension', 'is_ffpe', 'karnofsky_performance_score', 'lactate_dehydrogenase_result', 'laterality', 'longes

### Step 2: Find Candidate Demographic Features

In [3]:
# Identifying candidate age and gender columns from clinical data columns
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth']
candidate_gender_cols = ['gender']

# Loading clinical data to preview candidate columns
clinical_file_path, _ = tcga_get_relevant_filepaths(os.path.join(tcga_root_dir, 'TCGA_Kidney_Papillary_Cell_Carcinoma_(KIRP)'))
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)

# Extract and preview age columns
age_preview = {}
for col in candidate_age_cols:
    if col in clinical_df.columns:
        age_preview[col] = clinical_df[col].head(5).tolist()

print("Age columns preview:")
print(age_preview)

# Extract and preview gender columns
gender_preview = {}
for col in candidate_gender_cols:
    if col in clinical_df.columns:
        gender_preview[col] = clinical_df[col].head(5).tolist()

print("Gender columns preview:")
print(gender_preview)


Age columns preview:
{'age_at_initial_pathologic_diagnosis': [53.0, 71.0, 71.0, 67.0, 80.0], 'days_to_birth': [-19603.0, -26244.0, -26134.0, -24626.0, -29275.0]}
Gender columns preview:
{'gender': ['MALE', 'MALE', 'FEMALE', 'MALE', 'MALE']}


### Step 3: Select Demographic Features

In [4]:
# Step: Select Demographic Features

# Selecting age column
age_columns = {'age_at_initial_pathologic_diagnosis': [53.0, 71.0, 71.0, 67.0, 80.0], 
               'days_to_birth': [-19603.0, -26244.0, -26134.0, -24626.0, -29275.0]}

# Examine age columns
# 'age_at_initial_pathologic_diagnosis' contains direct age values (in years)
# 'days_to_birth' contains negative values representing days before birth (more complex to interpret)
# Choose 'age_at_initial_pathologic_diagnosis' as it directly represents age in years
age_col = 'age_at_initial_pathologic_diagnosis'

# Selecting gender column
gender_columns = {'gender': ['MALE', 'MALE', 'FEMALE', 'MALE', 'MALE']}

# There's only one gender column and it contains valid values (MALE/FEMALE)
gender_col = 'gender'

# Print the chosen columns
print(f"Selected age column: {age_col}")
print(f"Age column preview: {age_columns[age_col]}")
print(f"Selected gender column: {gender_col}")
print(f"Gender column preview: {gender_columns[gender_col]}")


Selected age column: age_at_initial_pathologic_diagnosis
Age column preview: [53.0, 71.0, 71.0, 67.0, 80.0]
Selected gender column: gender
Gender column preview: ['MALE', 'MALE', 'FEMALE', 'MALE', 'MALE']


### Step 4: Feature Engineering and Validation

In [5]:
# 1. Extract and standardize clinical features
# Use tcga_select_clinical_features which will automatically create the trait variable and add age/gender if provided
# Use the correct cohort identified in Step 1
cohort_dir = os.path.join(tcga_root_dir, 'TCGA_Kidney_Papillary_Cell_Carcinoma_(KIRP)')
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)

# Load the clinical data if not already loaded
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)

linked_clinical_df = tcga_select_clinical_features(
    clinical_df, 
    trait=trait, 
    age_col=age_col, 
    gender_col=gender_col
)

# Print preview of clinical features
print("Clinical features (first 5 rows):")
print(linked_clinical_df.head())

# 2. Process gene expression data
print("\nProcessing gene expression data...")
# Load genetic data from the same cohort directory
genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)

# Check gene data shape
print(f"Original gene data shape: {genetic_df.shape}")

# Save a version of the gene data before normalization (as a backup)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
genetic_df.to_csv(out_gene_data_file.replace('.csv', '_original.csv'))

# We need to transpose genetic data so genes are rows and samples are columns for normalization
gene_df_for_norm = genetic_df.copy()  # Keep original orientation for now

# Try to normalize gene symbols - adding debug output to understand what's happening
print("Attempting to normalize gene symbols...")
try:
    # First check if we need to transpose based on the data format
    # In TCGA data, typically genes are rows and samples are columns
    if gene_df_for_norm.shape[0] > gene_df_for_norm.shape[1]:
        # More rows than columns, likely genes are rows already
        normalized_gene_df = normalize_gene_symbols_in_index(gene_df_for_norm)
    else:
        # Need to transpose first
        normalized_gene_df = normalize_gene_symbols_in_index(gene_df_for_norm.T)
        
    print(f"Gene data shape after normalization: {normalized_gene_df.shape}")
    
    # Check if normalization returned empty DataFrame
    if normalized_gene_df.shape[0] == 0:
        print("WARNING: Gene symbol normalization returned an empty DataFrame.")
        print("Using original gene data instead of normalized data.")
        # Use original data
        normalized_gene_df = genetic_df
    
except Exception as e:
    print(f"Error during gene symbol normalization: {e}")
    print("Using original gene data instead.")
    normalized_gene_df = genetic_df

# Save gene data
normalized_gene_df.to_csv(out_gene_data_file)
print(f"Gene data saved to: {out_gene_data_file}")

# 3. Link clinical and genetic data
# TCGA data uses the same sample IDs in both datasets
print("\nLinking clinical and genetic data...")
print(f"Clinical data shape: {linked_clinical_df.shape}")
print(f"Genetic data shape: {normalized_gene_df.shape}")

# Find common samples between clinical and genetic data
# In TCGA, samples are typically columns in the gene data and index in the clinical data
common_samples = set(linked_clinical_df.index).intersection(set(normalized_gene_df.columns))
print(f"Number of common samples: {len(common_samples)}")

if len(common_samples) == 0:
    print("ERROR: No common samples found between clinical and genetic data.")
    # Try the alternative orientation
    common_samples = set(linked_clinical_df.index).intersection(set(normalized_gene_df.index))
    print(f"Checking alternative orientation: {len(common_samples)} common samples found.")
    
    if len(common_samples) == 0:
        # Use is_final=False mode which doesn't require df and is_biased
        validate_and_save_cohort_info(
            is_final=False,
            cohort="TCGA",
            info_path=json_path,
            is_gene_available=True,
            is_trait_available=True
        )
        print("The dataset was determined to be unusable for this trait due to no common samples. No data files were saved.")
else:
    # Filter clinical data to only include common samples
    linked_clinical_df = linked_clinical_df.loc[list(common_samples)]
    
    # Create linked data by merging
    linked_data = pd.concat([linked_clinical_df, normalized_gene_df[list(common_samples)].T], axis=1)
    
    print(f"\nLinked data shape: {linked_data.shape}")
    print("Linked data preview (first 5 rows, first few columns):")
    display_cols = [trait, 'Age', 'Gender'] + list(linked_data.columns[3:5])
    print(linked_data[display_cols].head())
    
    # 4. Handle missing values
    linked_data = handle_missing_values(linked_data, trait)
    print(f"\nData shape after handling missing values: {linked_data.shape}")
    
    # 5. Check for bias in features
    print("\nChecking for bias in features:")
    is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    
    # 6. Validate and save cohort info
    print("\nPerforming final validation...")
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort="TCGA",
        info_path=json_path,
        is_gene_available=len(linked_data.columns) > 3,  # More than just trait/age/gender columns
        is_trait_available=trait in linked_data.columns,
        is_biased=is_trait_biased,
        df=linked_data,
        note="Data from TCGA Kidney Papillary Cell Carcinoma cohort used for chronic kidney disease analysis."
    )
    
    # 7. Save linked data if usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to: {out_data_file}")
        
        # Also save clinical data separately
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_columns = [col for col in linked_data.columns if col in [trait, 'Age', 'Gender']]
        linked_data[clinical_columns].to_csv(out_clinical_data_file)
        print(f"Clinical data saved to: {out_clinical_data_file}")
    else:
        print("The dataset was determined to be unusable for this trait. No data files were saved.")

Clinical features (first 5 rows):
                 Chronic_kidney_disease   Age  Gender
sampleID                                             
TCGA-2K-A9WE-01                       1  53.0       1
TCGA-2Z-A9J1-01                       1  71.0       1
TCGA-2Z-A9J2-01                       1  71.0       0
TCGA-2Z-A9J3-01                       1  67.0       1
TCGA-2Z-A9J5-01                       1  80.0       1

Processing gene expression data...


Original gene data shape: (20530, 323)


Attempting to normalize gene symbols...
Gene data shape after normalization: (19848, 323)


Gene data saved to: ../../output/preprocess/Chronic_kidney_disease/gene_data/TCGA.csv

Linking clinical and genetic data...
Clinical data shape: (352, 3)
Genetic data shape: (19848, 323)
Number of common samples: 323

Linked data shape: (323, 19851)
Linked data preview (first 5 rows, first few columns):
                 Chronic_kidney_disease   Age  Gender      A1BG  A1BG-AS1
TCGA-5P-A9KE-01                       1  70.0       1 -1.832274 -2.060683
TCGA-B9-7268-01                       1  59.0       1 -2.074374 -2.547183
TCGA-BQ-5879-01                       1  32.0       0  0.389126  0.522517
TCGA-P4-A5ED-01                       1  51.0       1  1.791126  1.324417
TCGA-Y8-A8RY-01                       1  63.0       1 -0.335474 -0.016783



Data shape after handling missing values: (323, 19851)

Checking for bias in features:
For the feature 'Chronic_kidney_disease', the least common label is '0' with 32 occurrences. This represents 9.91% of the dataset.
The distribution of the feature 'Chronic_kidney_disease' in this dataset is fine.

Quartiles for 'Age':
  25%: 54.0
  50% (Median): 61.459375
  75%: 71.0
Min: 28.0
Max: 88.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0' with 87 occurrences. This represents 26.93% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.


Performing final validation...


Linked data saved to: ../../output/preprocess/Chronic_kidney_disease/TCGA.csv
Clinical data saved to: ../../output/preprocess/Chronic_kidney_disease/clinical_data/TCGA.csv
